## 전처리 py 파일 내장 함수별 오류 점검

In [18]:
import pandas as pd
import numpy as np
import re
from SamsungPhone import assign_samsung_product_name
from AppleIphone import assign_iphone_product_name
from RemainedPhone import assign_remained_product_name
import basicfunction as bf

class DataPreprocessor:
    '''
    # 크롤링 통해 처음 들어오는 열 : 
    pid	(상품번호) -> 중복값 제거 후 삭제
    name(게시글제목) -> product_name 출력 후 제거
    description(게시글 내용) -> product_name 출력 후 제거
    price(종속변수이자, 중고가)
    time(게시글 등록 시간 : 몇 분 전, 몇 시간 전 등) -> hours(시간 단위 통일화) 로 대체
    quality(제품 등급) #영호가 전달한 등급 기반으로 처리
    favoritecount(찜, 좋아요)
    viewcount(조회수)
    address(거래 주소) #삭제
    keyword(키워드) # 삭제	
    crawling(크롤링한 웹페이지)
    created_at(크롤링한 시간) # 동일한 날에 진행하여 완료한다는 가정 (삭제)
    
    # 추가해야할 열
    product_name(상품명)
    release_year(출시년도)
    screen_size(화면 인치)
    cost(원가)
    capacity(메모리 용량)
    ram(램)
    weight(무게)
    화소별 카메라 개수 : back_camera_number_1, back_camera_number_2, back_camera_number_3
    카메라별 화소 : back_camera_1, back_camera_2, back_camera_3		
    Max_video_playtime(최대 비디오 재생시간)
    Max_audio_playtime(최대 오디오 재생시간)
    battery_capacity(배터리 용량)
    '''
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)
        self.prepro_df = self.df.copy()
    
    def add_basic_and_remove_irrelevant_products(self):
        '''
        필요없는 제품 제거
        '''
        # 필요 없는 열 삭제
        self.prepro_df.drop(['address', 'keyword'], axis=1, inplace=True)

        # 필요한 열 추가
        self.prepro_df['product_name'] = np.nan
        self.prepro_df['release_year'] = np.nan
        self.prepro_df['screen_size'] = np.nan
        self.prepro_df['cost'] = np.nan
        self.prepro_df['capacity'] = np.nan
        self.prepro_df['ram'] = np.nan
        self.prepro_df['weight'] = np.nan
        self.prepro_df['back_camera_number_1'] = np.nan
        self.prepro_df['back_camera_number_2'] = np.nan
        self.prepro_df['back_camera_number_3'] = np.nan
        self.prepro_df['back_camera_1'] = np.nan
        self.prepro_df['back_camera_2'] = np.nan
        self.prepro_df['back_camera_3'] = np.nan
        self.prepro_df['Max_video_playtime'] = np.nan
        self.prepro_df['Max_audio_playtime'] = np.nan
        self.prepro_df['battery_capacity'] = np.nan

        # 모두 소문자로 변경
        self.prepro_df['description'] = self.prepro_df['description'].str.lower()
        self.prepro_df['name'] = self.prepro_df['name'].str.lower()

        self.prepro_df = self.prepro_df[~self.prepro_df['name'].str.contains('아이패드|에어팟|디지털기기|구매합니다|아이폰x|교환|케어|교신|에디션|톰')]
        self.prepro_df = self.prepro_df[~self.prepro_df['name'].str.contains('갤럭시s23울트라폴드4플립4|아이폰13미니아이폰14아이폰14프로아이폰14프로맥스미개봉새상퓸')]
        self.prepro_df = self.prepro_df[~self.prepro_df['name'].str.contains('애플아이폰14프로맥스아이폰14프로아이폰14아이폰13미니새상품')]
        self.prepro_df['description'] = self.prepro_df['description'].fillna("")
        self.prepro_df = self.prepro_df[~self.prepro_df['description'].str.contains('아이패드|에어팟|디지털기기|구매합니다|아이폰X|교환|교신|폰고래|에디션|톰')]
    
    # 시간 통일화
    def assign_h_time(self):
        '''
         time 열을 시간 단위로 통일화하여 hours 열에 할당
         실제 실행될 때 크롤링 일자가 모두 동일하다는 가정 하에 진행한 코드입니다.
        '''
        self.prepro_df['hours'] = np.nan
        self.prepro_df['hours'] = self.prepro_df['time'].str.extract('(\d+)\s*시간', expand=False).astype(float)
        self.prepro_df.loc[self.prepro_df['time'].str.contains('분'), 'hours'] = self.prepro_df['time'].str.extract('(\d+)\s*분', expand=False).astype(float) / 60
        self.prepro_df.loc[self.prepro_df['time'].str.contains('일'), 'hours'] = self.prepro_df['time'].str.extract('(\d+)\s*일', expand=False).astype(float) * 24
        self.prepro_df.loc[self.prepro_df['time'].str.contains('달'), 'hours'] = self.prepro_df['time'].str.extract('(\d+)\s*달', expand=False).astype(float) * 720
        self.prepro_df.loc[self.prepro_df['time'].str.contains('년'), 'hours'] = self.prepro_df['time'].str.extract('(\d+)\s*년', expand=False).astype(float) * 8640
    
        # # hours의 nan값 삭제
        # mismatch = self.prepro_df[self.prepro_df['hours'].isna() != self.prepro_df['product_name'].isna()]
        # self.prepro_df = self.prepro_df.drop(mismatch[self.prepro_df['hours'].isna()].index)

    # 중복게시글 제거
    def remove_duplicates_and_unnecessary_columns(self):
        '''
        중복된 게시글 제거 (pid 열 기준) : 중복된 게시글의 최근 게시글만 남기고 삭제
        이후 필요없는 행 제거
        '''
        self.prepro_df = self.prepro_df.sort_values('hours').drop_duplicates('pid', keep='first')
        self.prepro_df.drop(['time', 'pid'], axis=1, inplace=True)

    # 제품명과 용량 추출 
    def assign_product_name(self):
        '''
        name과 description에서 제품명 추출
        '''
        # name열에서 '1tb'를 '1000'으로 변경
        self.prepro_df['name'] = self.prepro_df['name'].apply(bf.BasicFunctions.convert_tb_to_gb)
        
        # description열에서 '1tb'를 '1000'으로 변경
        self.prepro_df['description'] = self.prepro_df['description'].apply(bf.BasicFunctions.convert_tb_to_gb)

        # name 열과 description 열에서 용량 추출 : name_capacity열, description_capacity열
        self.prepro_df['name_capacity'] = self.prepro_df['name'].apply(bf.BasicFunctions.extract_capacity)
        self.prepro_df['description_capacity'] = self.prepro_df['description'].apply(lambda x: bf.BasicFunctions.extract_capacity(str(x)) if pd.notnull(x) else None)

        # product_name, capacity 추출
        self.prepro_df = assign_iphone_product_name(self.prepro_df) # 아이폰
        self.prepro_df = assign_samsung_product_name(self.prepro_df) # 갤럭시
        
        # 아이폰과 갤럭시에서 추출된 내용 외의 것 제거 (1차)
        filter_words = ['mini', '미니', 'pro', '프로','max', '맥스', '플러스', '//+', '울트라', 'ultra']
        self.prepro_df = self.prepro_df[~(self.prepro_df['product_name'].isna() & self.prepro_df['name'].apply(lambda x: any(word in x for word in filter_words)))]
        self.prepro_df = self.prepro_df[~(self.prepro_df['product_name'].isna() & self.prepro_df['description'].apply(lambda x: any(word in str(x) for word in filter_words)))]
 
        # 나머지 (아이폰11, 12, 13, 14 & 갤럭시 s20, 21, 22, 23)
        self.prepro_df = assign_remained_product_name(self.prepro_df)

    # 사양 연결
    def map_phone_info(self):
        '''
        1. product_name nan값 삭제
        2. capacity 열에 데이터 최종 입력
            1) capacity가 nan인 값인 행 & name_capacity, description_capacity열 중 데이터가 한 열에만 존재할 경우
            2) capacity가 nan인 값인 행 & name_capacity, description_capacity열 모두 데이터가 있을 경우
                => name_capacity의 값을 우선적으로 capacity에 추가
            3) capacity가 nan 값인 행 삭제 (nan 값이 없게끔 크로스 삭제)
        3. name_capacity, description_capacity 열 삭제
        4. product_name 기준으로 사양들 맵핑
        '''
        # 1. product_name nan값 삭제
        self.prepro_df = self.prepro_df[~(self.prepro_df['product_name'].isna())]

        # 2. capacity 열에 데이터 최종 입력 
        # 1) 값이 한 열에만 있을 경우 & 2) 두 열 모두 값이 있을 경우 
        condition = (self.prepro_df['capacity'].isna()) & \
                    (self.prepro_df['name_capacity'].notna() | self.prepro_df['description_capacity'].notna())
       
        # name_capacity의 값을 먼저 선택 -> 그래도 nan이면 fillna를 사용해 nan값을 description_capacity로 대체
        self.prepro_df.loc[condition, 'capacity'] = self.prepro_df.loc[condition, 'name_capacity'].fillna(self.prepro_df['description_capacity'])

        # 3) capacity가 nan 값인 행 삭제
        self.prepro_df = self.prepro_df[~(self.prepro_df['capacity'].isna())]

        # 3. name_capacity, description_capacity 삭제
        self.prepro_df.drop(['name_capacity', 'description_capacity'], axis=1, inplace=True)
        
        # capacity 열 타입 변환(object -> int)
        self.prepro_df['capacity'] = self.prepro_df['capacity'].astype(int)

        # 4. product_name 기준으로 사양들 맵핑
        phone_info = pd.read_csv('phone_info_v2.csv')
        phone_info = phone_info.drop_duplicates(subset=['product', 'capacity'])
        
        # product, capacity 열을 인덱스로 설정
        phone_info = phone_info.set_index(['product', 'capacity']) 

        # prepro_df의 각 열을 phone_info 데이터프레임을 사용하여 업데이트
        for col in ['release_year', 'screen_size', 'weight', 'back_camera_1',
                    'back_camera_number_1', 'back_camera_2', 'back_camera_number_2',
                    'back_camera_3', 'back_camera_number_3', 'Max_video_playtime',
                    'Max_audio_playtime', 'battery_capacity', 'ram', 'cost']:
            
            # 'product_name'과 'capacity' 열을 zip으로 묶어서 매핑
            self.prepro_df[col] = list(map(lambda x: phone_info.loc[(x[0], x[1]), col] 
                                           if (x[0], x[1]) in phone_info.index else np.nan, 
                                           zip(self.prepro_df['product_name'], self.prepro_df['capacity'])))

        # 사양이 없는 행 제거
        self.prepro_df = self.prepro_df.dropna(subset=['release_year'])

    # 타입 변환
    def change_data_type(self):
        self.prepro_df['cost'] = self.prepro_df['cost'].str.replace(',', '').astype(int)
        self.prepro_df['ram'] = self.prepro_df['ram'].astype(int)
        self.prepro_df['weight'] = self.prepro_df['weight'].astype(float)
        self.prepro_df['back_camera_number_1'] = self.prepro_df['back_camera_number_1'].astype(int)
        self.prepro_df['back_camera_number_2'] = self.prepro_df['back_camera_number_2'].astype(int)
        self.prepro_df['back_camera_number_3'] = self.prepro_df['back_camera_number_3'].astype(int)
        self.prepro_df['back_camera_1'] = self.prepro_df['back_camera_1'].str.replace(',', '').astype(int)
        self.prepro_df['back_camera_2'] = self.prepro_df['back_camera_2'].str.replace(',', '').astype(int)
        self.prepro_df['back_camera_3'] = self.prepro_df['back_camera_3'].str.replace(',', '').astype(int)
        self.prepro_df['Max_video_playtime'] = self.prepro_df['Max_video_playtime'].astype(int)
        self.prepro_df['Max_audio_playtime'] = self.prepro_df['Max_audio_playtime'].astype(int)
        self.prepro_df['battery_capacity'] = self.prepro_df['battery_capacity'].str.replace(',', '').astype(int)

    # 출시 년도 -> 년식으로 변경
    def convert_as_used_year(self):
        '''
        2023년 대비 년식을 구하는 함수
        '''
        self.prepro_df['used_year'] = self.prepro_df['release_year'].apply(lambda x: 2023 - x)
        
        # # release_year 열 삭제
        # self.prepro_df.drop(['release_year'], axis=1, inplace=True)

    # 등급 변환
    def extract_and_map_quality(self):
        '''
        description으로부터 등급 정보를 추출하고 점수로 맵핑하는 함수
        '''
        # 등급 추출 함수
        def extract_grade_desc(s):
            # "등급 :" 뒤에 공백 상관없이 오는 영문 찾기
            grade = re.search(r'등급 :(\w+)', s)
            if grade is not None and any(letter in grade.group(1).lower() for letter in ['s', 'b', 'a']):
                return grade.group(1) # group(1) : 첫 번째 그룹
            
            # "등급 :"에 해당하지 않는다면, "급" 앞에 공백 상관없이 있는 영문 찾기
            grade = re.search(r'(\w+)급', s)
            if grade is not None and any(letter in grade.group(1).lower() for letter in ['s', 'b', 'a']):
                return grade.group(1)
            return np.nan 

        grade_filter = ~self.prepro_df['quality'].isin(['미개봉', 'S급', 'A급', 'B급'])
        self.prepro_df.loc[grade_filter, 'quality'] = self.prepro_df.loc[grade_filter, 'description'].apply(extract_grade_desc)

        # 등급 재추출 및 정의
        self.prepro.loc[self.prepro['quality'].isin(['a', 'aaa']), 'quality'] = 'A급'
        self.prepro.loc[self.prepro['quality'].isin(['b']), 'quality'] = 'B급'
        self.prepro.loc[self.prepro['quality'].isin(['c']), 'quality'] = 'C급'

        # 등급 점수화
        quality_mapping = {
            '미개봉': 5,
            'S급': 4,
            'A급': 3,
            'B급': 2,
            'C급': 1,
        }
        self.prepro_df['quality'] = self.prepro_df['quality'].map(quality_mapping)

        # 등급이 없는 행 삭제
        self.prepro_df = self.prepro_df.dropna(subset=['quality'])

        # 필요없는 행 삭제
        self.prepro_df.drop(['name', 'description'], axis=1, inplace=True)

    # 조회수, 찜(좋아요) 가중치
    def normalize_viewcount_favoritecount(self):
        '''
        웹페이지 사용자 수를 기준으로 'viewcount'와 'favoritecount' 열을 정규화하는 함수
        '''
        user_site = {
            0: 4640000,  # 당근마켓
            2: 490000,   # 번개장터
            1: 120000,   # 중고나라
        }

        for site, user_count in user_site.items():
            self.prepro_df.loc[self.prepro_df['crawling'] == site, 'viewcount'] /= user_count
            self.prepro_df.loc[self.prepro_df['crawling'] == site, 'favoritecount'] /= user_count
    
        # nan 값은 0으로 변환
        self.prepro_df['viewcount'].fillna(0, inplace=True)
        self.prepro_df['favoritecount'].fillna(0, inplace=True)

        # 필요없는 열 삭제
        self.prepro_df.drop(['crawling', 'created_at'], axis=1, inplace=True)

    def execute(self):
        '''
        앞서 정의한 모든 전처리 메소드를 순차적으로 실행
        '''
        self.add_basic_and_remove_irrelevant_products()
        self.assign_h_time()
        self.remove_duplicates_and_unnecessary_columns()
        self.assign_product_name()
        self.map_phone_info()
        self.change_data_type()   
        self.convert_as_used_year()
        # self.extract_and_map_quality()
        # self.normalize_viewcount_favoritecount()
        
        return self.prepro_df

In [19]:
test = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
test_df = test.execute()
test_df.head(3)

,name,description,price,quality,favoritecount,viewcount,crawling,created_at,product_name,release_year,...,back_camera_number_2,back_camera_number_3,back_camera_1,back_camera_2,back_camera_3,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,아이폰12미니풀박.256gb배터리효율100,,450000,B급,NaN,NaN,1,01:04.1,아이폰 12 Mini,2020.0,...,0,0,1200,0,0,10,50,2227,0.016667,3.0
9748,갤럭시z폴드4256베이지,"안녕하세요, 갤럭시 z폴드4 256 베이지 판매합니다.\n자급제 단말입니다.\n\n...",970000,B급,0.0,0.0,2,22:46.3,갤럭시 Z 폴드4,2022.0,...,1,1,1200,5000,1000,20,72,4400,0.016667,1.0
21572,아이폰12128g블랙a급!,아이폰12입니다\n\n128g 블랙색상이구여\n\n외관 테두리 상단에 기스 조금있고...,530000,A급,0.0,37.0,1,38:44.9,아이폰 12,2020.0,...,0,0,1200,0,0,11,65,2815,0.016667,3.0


In [20]:
test_df.loc[test_df['used_year'].isna(), ['release_year', 'used_year']]

,release_year,used_year


In [22]:
test_df['used_year'].unique()

array([3., 1., 0., 2., 4.])

In [42]:
test_df.loc[test_df['release_year'].isna(), ['product_name', 'capacity']]

,product_name,capacity


In [5]:
test_df['quality'].unique()

array([2., 3., 4., 5.])

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9329 entries, 8137 to 11048
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 9329 non-null   int64  
 1   quality               9329 non-null   float64
 2   favoritecount         9329 non-null   float64
 3   viewcount             9329 non-null   float64
 4   product_name          9329 non-null   object 
 5   screen_size           9329 non-null   float64
 6   cost                  9329 non-null   object 
 7   capacity              9329 non-null   int32  
 8   ram                   9329 non-null   float64
 9   weight                9329 non-null   float64
 10  back_camera_number_1  9329 non-null   float64
 11  back_camera_number_2  9329 non-null   float64
 12  back_camera_number_3  9329 non-null   float64
 13  back_camera_1         9329 non-null   object 
 14  back_camera_2         9329 non-null   object 
 15  back_camera_3    

In [35]:
import pandas as pd
import numpy as np
p_info = pd.read_csv("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_info_v2.csv")
p_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product               89 non-null     object 
 1   release_year          89 non-null     int64  
 2   screen_size           89 non-null     float64
 3   weight                89 non-null     float64
 4   back_camera_1         89 non-null     object 
 5   back_camera_number_1  89 non-null     int64  
 6   back_camera_2         89 non-null     object 
 7   back_camera_number_2  89 non-null     int64  
 8   back_camera_3         89 non-null     object 
 9   back_camera_number_3  89 non-null     int64  
 10  Max_video_playtime    89 non-null     int64  
 11  Max_audio_playtime    89 non-null     int64  
 12  battery_capacity      89 non-null     object 
 13  ram                   89 non-null     int64  
 14  capacity              89 non-null     int64  
 15  cost                  89 

## 전처리 파일 2차 실행 (스케일링 전)

In [1]:
from phone_CrawlingDataPreprocessor import DataPreprocessor
prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
prepro_df = prepro.execute()
prepro_df

,price,quality,favoritecount,viewcount,product_name,screen_size,cost,capacity,ram,weight,...,back_camera_number_2,back_camera_number_3,back_camera_1,back_camera_2,back_camera_3,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,450000,2.0,0.0,0.000000,아이폰 12 Mini,13.70,1160000,256,4,0.133,...,0,0,1200,0,0,10,50,2227,0.016667,3.0
9748,970000,2.0,0.0,0.000000,갤럭시 Z 폴드4,19.21,1998700,256,12,0.263,...,1,1,1200,5000,1000,20,72,4400,0.016667,1.0
21572,530000,3.0,0.0,0.000308,아이폰 12,15.40,1020000,128,4,0.162,...,0,0,1200,0,0,11,65,2815,0.016667,3.0
19929,950000,4.0,0.0,0.000002,갤럭시 S 23 플러스,16.65,1353000,256,8,0.195,...,1,1,1200,1000,5000,27,84,4700,0.033333,0.0
22773,560000,3.0,0.0,0.000217,아이폰 13 Mini,13.70,950000,128,4,0.140,...,0,0,1200,0,0,13,55,2406,0.050000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,750000,4.0,0.0,0.000053,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,...,0,0,1200,0,0,18,83,4500,7920.000000,3.0
9744,750000,2.0,0.0,0.000161,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,...,0,0,1200,0,0,18,83,4500,8640.000000,3.0
12807,1050000,2.0,0.0,0.000308,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,...,0,0,1200,0,0,18,83,4500,8640.000000,3.0
12796,1200000,2.0,0.0,0.000092,갤럭시 Z 폴드3,19.19,1998700,256,12,0.271,...,0,0,1200,0,0,18,63,4400,8640.000000,2.0


In [7]:
# 타입 수정 후
prepro_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9329 entries, 8137 to 11048
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 9329 non-null   int64  
 1   quality               9329 non-null   float64
 2   favoritecount         9329 non-null   float64
 3   viewcount             9329 non-null   float64
 4   product_name          9329 non-null   object 
 5   screen_size           9329 non-null   float64
 6   cost                  9329 non-null   int32  
 7   capacity              9329 non-null   int32  
 8   ram                   9329 non-null   int32  
 9   weight                9329 non-null   float64
 10  back_camera_number_1  9329 non-null   int32  
 11  back_camera_number_2  9329 non-null   int32  
 12  back_camera_number_3  9329 non-null   int32  
 13  back_camera_1         9329 non-null   int32  
 14  back_camera_2         9329 non-null   int32  
 15  back_camera_3    

In [2]:
# 타입 수정 전
prepro_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9329 entries, 8137 to 11048
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 9329 non-null   int64  
 1   quality               9329 non-null   float64
 2   favoritecount         9329 non-null   float64
 3   viewcount             9329 non-null   float64
 4   product_name          9329 non-null   object 
 5   screen_size           9329 non-null   float64
 6   cost                  9329 non-null   object 
 7   capacity              9329 non-null   int32  
 8   ram                   9329 non-null   float64
 9   weight                9329 non-null   float64
 10  back_camera_number_1  9329 non-null   float64
 11  back_camera_number_2  9329 non-null   float64
 12  back_camera_number_3  9329 non-null   float64
 13  back_camera_1         9329 non-null   object 
 14  back_camera_2         9329 non-null   object 
 15  back_camera_3    

## 스케일링 py 파일 실행 점검

In [8]:
from phone_Modeling_Preprocessor import modeling_Preprocessor
scaled_df = modeling_Preprocessor(prepro_df)
scaled_df

,price,quality,favoritecount,viewcount,screen_size,cost,capacity,ram,weight,back_camera_number_1,back_camera_number_2,back_camera_number_3,back_camera_1,back_camera_2,back_camera_3,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,450000,0.693147,0.0,0.000000,0.291289,0.429971,0.504314,0.171856,0.000000,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.183447,0.574961,0.199267,0.000000,0.792481
9748,970000,0.693147,0.0,0.000000,0.990334,0.833867,0.504314,0.828144,0.907189,0.00000,0.908879,1.0,0.0,0.952448,0.821802,0.753284,0.769301,0.873439,0.000000,0.000000
21572,530000,1.098612,0.0,0.052656,0.533182,0.334492,0.252157,0.171856,0.262451,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.261801,0.714791,0.431242,0.000000,0.792481
19929,950000,1.386294,0.0,0.000338,0.694572,0.544222,0.504314,0.585928,0.509143,0.00000,0.908879,1.0,0.0,0.853070,0.917538,1.000000,0.851457,0.938740,0.052677,NaN
22773,560000,1.098612,0.0,0.038424,0.291289,0.281714,0.252157,0.171856,0.068249,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.399137,0.625758,0.275807,0.083491,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,750000,1.386294,0.0,0.010130,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,0.993387,0.792481
9744,750000,0.693147,0.0,0.029286,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,1.000000,0.792481
12807,1050000,0.693147,0.0,0.052630,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,1.000000,0.792481
12796,1200000,0.693147,0.0,0.017214,0.988180,0.833867,0.504314,0.828144,0.947059,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.698134,0.873439,1.000000,0.500000


## 전처리 파일 최종본 실행

In [1]:
from phone_CrawlingDataPreprocessor import DataPreprocessor
prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
prepro_df = prepro.execute()
prepro_df

,price,quality,favoritecount,viewcount,screen_size,cost,capacity,ram,weight,back_camera_number_1,back_camera_number_2,back_camera_number_3,back_camera_1,back_camera_2,back_camera_3,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,450000,0.000000,0.0,0.000000,0.291289,0.429971,0.504314,0.171856,0.000000,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.183447,0.574961,0.199267,0.000000,0.965326
9748,970000,0.000000,0.0,0.000000,0.990334,0.833867,0.504314,0.828144,0.907189,0.00000,0.908879,1.0,0.0,0.952448,0.821802,0.753284,0.769301,0.873439,0.000000,0.832952
21572,530000,0.442507,0.0,0.052656,0.533182,0.334492,0.252157,0.171856,0.262451,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.261801,0.714791,0.431242,0.000000,0.965326
19929,950000,0.756471,0.0,0.000338,0.694572,0.544222,0.504314,0.585928,0.509143,0.00000,0.908879,1.0,0.0,0.853070,0.917538,1.000000,0.851457,0.938740,0.052677,0.000000
22773,560000,0.442507,0.0,0.038424,0.291289,0.281714,0.252157,0.171856,0.068249,0.63093,0.000000,0.0,0.0,0.000000,0.000000,0.399137,0.625758,0.275807,0.083491,0.916461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,750000,0.756471,0.0,0.010130,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,0.993387,0.965326
9744,750000,0.000000,0.0,0.029286,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,1.000000,0.965326
12807,1050000,0.000000,0.0,0.052630,1.000000,0.969077,0.504314,0.828144,1.000000,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.845074,0.895688,1.000000,0.965326
12796,1200000,0.000000,0.0,0.017214,0.988180,0.833867,0.504314,0.828144,0.947059,1.00000,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.698134,0.873439,1.000000,0.916461


In [2]:
prepro_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9331 entries, 8137 to 11048
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price                 9331 non-null   int64  
 1   quality               9331 non-null   float64
 2   favoritecount         9331 non-null   float64
 3   viewcount             9331 non-null   float64
 4   screen_size           9331 non-null   float64
 5   cost                  9331 non-null   float64
 6   capacity              9331 non-null   float64
 7   ram                   9331 non-null   float64
 8   weight                9331 non-null   float64
 9   back_camera_number_1  9331 non-null   float64
 10  back_camera_number_2  9331 non-null   float64
 11  back_camera_number_3  9331 non-null   float64
 12  back_camera_1         9331 non-null   float64
 13  back_camera_2         9331 non-null   float64
 14  back_camera_3         9331 non-null   float64
 15  Max_video_playtim

## 데이터 셋 분류

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np

# 평가 지표
# ★ 평균 제곱 오차(MSE)  → 이상치가 적은 경우에는 MSE가 더 적합
# ★ 결정 계수(R2)

# n_estimators : 생성할 트리 개수

In [4]:
# 학습 데이터, 테스트 데이터 분리
x = prepro_df.loc[:, prepro_df.columns != 'price']
y = prepro_df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

## XGBoost 를 이용한 가격 예측

In [5]:
import optuna
from xgboost import XGBRegressor

def objectiveXGB(trial):
    # Define the search space for each parameter
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
    }

    # 모델 학습
    model = XGBRegressor(**params)
    model.fit(x_train, y_train)

    # 예측
    y_pred = model.predict(x_test)

    # 모델 평가    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"mse : {mse}")
    print(f"rmse : {rmse}")
    print(f"r2 : {r2}")
    
    return mse

# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study4 = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study4.optimize(objectiveXGB, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study4.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study4.best_trial.value, study4.best_trial.params))

[I 2023-06-21 01:29:42,877] A new study created in memory with name: no-name-e21a9196-60be-48a3-8917-b0bd8e5870fd
[I 2023-06-21 01:30:06,238] Trial 0 finished with value: 15573961149.121227 and parameters: {'n_estimators': 2586, 'learning_rate': 0.07900432889925468, 'max_depth': 10, 'min_child_weight': 8, 'gamma': 0.07747539415513222, 'subsample': 0.8859279792169359, 'colsample_bytree': 0.7667699345158958}. Best is trial 0 with value: 15573961149.121227.


mse : 15573961149.121227
rmse : 124795.67760592203
r2 : 0.8692885353236752


[I 2023-06-21 01:30:24,856] Trial 1 finished with value: 18418414227.022133 and parameters: {'n_estimators': 2030, 'learning_rate': 0.7882342549779966, 'max_depth': 10, 'min_child_weight': 1, 'gamma': 0.1808284817249028, 'subsample': 0.9677055769675824, 'colsample_bytree': 0.6336889424830798}. Best is trial 0 with value: 15573961149.121227.


mse : 18418414227.022133
rmse : 135714.45843027238
r2 : 0.8454151851556939


[I 2023-06-21 01:30:46,441] Trial 2 finished with value: 21129526368.032772 and parameters: {'n_estimators': 3582, 'learning_rate': 0.946394007838627, 'max_depth': 7, 'min_child_weight': 5, 'gamma': 0.21250580292099763, 'subsample': 0.9614348708708651, 'colsample_bytree': 0.9575226555922335}. Best is trial 0 with value: 15573961149.121227.


...

mse : 13459559975.203299
rmse : 116015.34370592237
r2 : 0.8870345969524309


[I 2023-06-21 01:39:41,291] Trial 98 finished with value: 14187661535.449492 and parameters: {'n_estimators': 1143, 'learning_rate': 0.05948138163627371, 'max_depth': 7, 'min_child_weight': 6, 'gamma': 0.3091456696308026, 'subsample': 0.6438043999110877, 'colsample_bytree': 0.7300440314037059}. Best is trial 21 with value: 12114723531.257904.


mse : 14187661535.449492
rmse : 119111.97057999457
r2 : 0.8809236775490994


[I 2023-06-21 01:39:50,751] Trial 99 finished with value: 15590688564.241447 and parameters: {'n_estimators': 1297, 'learning_rate': 0.09917904846430142, 'max_depth': 8, 'min_child_weight': 7, 'gamma': 0.3869094085723863, 'subsample': 0.5730645669716108, 'colsample_bytree': 0.8117035825353764}. Best is trial 21 with value: 12114723531.257904.


mse : 15590688564.241447
rmse : 124862.67882854928
r2 : 0.8691481429784216
Best trial: score 12114723531.257904, 
params {'n_estimators': 704, 'learning_rate': 0.012798552773938448, 'max_depth': 7, 'min_child_weight': 5, 'gamma': 0.3414618413108147, 'subsample': 0.5831546829963173, 'colsample_bytree': 0.5381338415408887}


In [6]:
# Best trial의 파라미터
best_params = study4.best_trial.params

# 최적의 모델 학습
best_xgb = XGBRegressor(n_estimators=best_params['n_estimators'],
                        learning_rate=best_params['learning_rate'],
                        max_depth=best_params['max_depth'],
                        min_child_weight=best_params['min_child_weight'],
                        gamma=best_params['gamma'],
                        subsample=best_params['subsample'],
                        colsample_bytree=best_params['colsample_bytree'],
                        random_state=42)
best_xgb.fit(x_train, y_train)

# 저장 경로 설정
import pickle
import os
save_path = 'XGBRegressor_best_model.pkl'

# 모델을 'XGBRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_xgb, f)

# 저장된 모델 로드
with open('XGBRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 12114723531.258
rmse : 110066.905
r2 : 0.898


## GradientBoosting 을 이용한 가격 예측

In [7]:
import optuna
from sklearn.ensemble import GradientBoostingRegressor

def objectiveGBR(trial):
    # Define the search space for each parameter
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 15),
        'max_features': trial.suggest_categorical('max_features', [1.0, 'sqrt', 'log2']),
        'random_state': 42,
    }

    # 모델 학습
    model = GradientBoostingRegressor(**params)
    model.fit(x_train, y_train)

    # 예측
    y_pred = model.predict(x_test)

    # 모델 평가    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"mse : {mse}")
    print(f"rmse : {rmse}")
    print(f"r2 : {r2}")
    
    return mse

# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study3 = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study3.optimize(objectiveGBR, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study3.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study3.best_trial.value, study3.best_trial.params))


[I 2023-06-21 01:41:40,275] A new study created in memory with name: no-name-5e32b751-5446-4644-a490-42f0c5b2c9c6
[I 2023-06-21 01:41:49,487] Trial 0 finished with value: 19946031144.148525 and parameters: {'n_estimators': 1009, 'learning_rate': 0.9027873492643156, 'max_depth': 9, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 19946031144.148525.


mse : 19946031144.148525
rmse : 141230.4186220112
r2 : 0.8325939739821198


[I 2023-06-21 01:42:14,484] Trial 1 finished with value: 13668364610.009518 and parameters: {'n_estimators': 3496, 'learning_rate': 0.09040162387004001, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: 13668364610.009518.


mse : 13668364610.009518
rmse : 116911.781313987
r2 : 0.8852821102609981


[I 2023-06-21 01:42:23,633] Trial 2 finished with value: 15711396922.822802 and parameters: {'n_estimators': 1853, 'learning_rate': 0.4976109749276875, 'max_depth': 3, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 1 with value: 13668364610.009518.


...

mse : 15157517845.741354
rmse : 123115.87162401667
r2 : 0.8727837228111888


[I 2023-06-21 02:07:29,171] Trial 98 finished with value: 12716801001.515142 and parameters: {'n_estimators': 2719, 'learning_rate': 0.030177265785921323, 'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 7 with value: 12181226636.545221.


mse : 12716801001.515142
rmse : 112768.7944491522
r2 : 0.8932685352820986


[I 2023-06-21 02:07:36,606] Trial 99 finished with value: 13616602728.86594 and parameters: {'n_estimators': 2435, 'learning_rate': 0.16515253507540958, 'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 7 with value: 12181226636.545221.


mse : 13616602728.86594
rmse : 116690.19979786623
r2 : 0.8857165450996302
Best trial: score 12181226636.545221, 
params {'n_estimators': 770, 'learning_rate': 0.0156752242832508, 'max_depth': 7, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


In [8]:
# Best trial의 파라미터
best_params = study3.best_trial.params

# 최적의 모델 학습
best_gbr = GradientBoostingRegressor(n_estimators=best_params['n_estimators'],
                                     learning_rate=best_params['learning_rate'],
                                     max_depth=best_params['max_depth'],
                                     min_samples_split=best_params['min_samples_split'],
                                     min_samples_leaf=best_params['min_samples_leaf'],
                                     max_features=best_params['max_features'],
                                     random_state=42)
best_gbr.fit(x_train, y_train)

# 저장 경로 설정
import pickle
import os
save_path = 'GradientBoostingRegressor_best_model.pkl'

# 모델을 'GradientBoostingRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_gbr, f)

# 저장된 모델 로드
with open('GradientBoostingRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 12181226636.545
rmse : 110368.594
r2 : 0.898


## CatBoostRegressor 를 이용한 가격 예측

In [13]:
import optuna
from catboost import CatBoostRegressor

def objectiveCatB(trial):
    # Define the search space for each parameter
    params = {
        'iterations': trial.suggest_int('iterations', 100, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
        'random_seed': 42,
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'verbose': False
    }
    # 손실함수와 평가지표를 RMSE로 하는 이유
    # (1) 예측 오차의 크기를 실제 값의 단위로 해석하고자 할 때 유용
    # (2) RMSE는 MSE와 동일한 오차 척도를 가지지만, 제곱근의 적용으로 오차의 크기를 실제 값의 단위로 변환
    #     이로 인해 RMSE는 예측 오차의 크기에 덜 민감하게 반응함
    
    # 모델 학습
    cat_features = ['quality'] # 범주형 데이터 'quality' : CatBoost의 cat_features 매개변수에 전달 필요
    catboost = CatBoostRegressor(**params)
    catboost.fit(x_train, y_train, eval_set=(x_test, y_test), early_stopping_rounds=50, verbose=False)
    
    # 예측
    y_pred = catboost.predict(x_test)
    
    # 모델 평가    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"mse : {mse}")
    print(f"rmse : {rmse}")
    print(f"r2 : {r2}")
    
    return mse

# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study2 = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study2.optimize(objectiveCatB, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study2.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study2.best_trial.value, study2.best_trial.params))

[I 2023-06-21 02:12:30,327] A new study created in memory with name: no-name-34300189-2b70-480a-a0a5-612457c2aeec
C:\Users\dohee\AppData\Local\Temp\ipykernel_21152\3506005532.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
[I 2023-06-21 02:12:33,048] Trial 0 finished with value: 12672865949.099916 and parameters: {'iterations': 2990, 'learning_rate': 0.040280796958740706, 'depth': 5, 'l2_leaf_reg': 0.000874029558520853}. Best is trial 0 with value: 12672865949.099916.


mse : 12672865949.099916
rmse : 112573.82444023085
r2 : 0.8936372799448622


C:\Users\dohee\AppData\Local\Temp\ipykernel_21152\3506005532.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
[I 2023-06-21 02:12:35,833] Trial 1 finished with value: 12108397026.133615 and parameters: {'iterations': 2768, 'learning_rate': 0.08928764634686989, 'depth': 9, 'l2_leaf_reg': 0.0002741389140746224}. Best is trial 1 with value: 12108397026.133615.


mse : 12108397026.133615
rmse : 110038.16168099872
r2 : 0.898374838936998


C:\Users\dohee\AppData\Local\Temp\ipykernel_21152\3506005532.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
[I 2023-06-21 02:12:37,622] Trial 2 finished with value: 12681246721.542765 and parameters: {'iterations': 2373, 'learning_rate': 0.028413605970284424, 'depth': 4, 'l2_leaf_reg': 2.1795453184529404e-05}. Best is trial 1 with value: 12108397026.133615.


...

mse : 12144440643.846132
rmse : 110201.81778830208
r2 : 0.8980723266847632


C:\Users\dohee\AppData\Local\Temp\ipykernel_21152\3506005532.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
[I 2023-06-21 02:24:48,105] Trial 98 finished with value: 12141873587.24966 and parameters: {'iterations': 3408, 'learning_rate': 0.030756108072516365, 'depth': 8, 'l2_leaf_reg': 1.5556388471962336e-05}. Best is trial 71 with value: 11898264128.231482.


mse : 12141873587.24966
rmse : 110190.17010264419
r2 : 0.8980938718603559


C:\Users\dohee\AppData\Local\Temp\ipykernel_21152\3506005532.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
[I 2023-06-21 02:25:33,233] Trial 99 finished with value: 12075467671.180283 and parameters: {'iterations': 1953, 'learning_rate': 0.0101951889796691, 'depth': 10, 'l2_leaf_reg': 9.45597357871608e-05}. Best is trial 71 with value: 11898264128.231482.


mse : 12075467671.180283
rmse : 109888.43283612831
r2 : 0.8986512133401177
Best trial: score 11898264128.231482, 
params {'iterations': 3847, 'learning_rate': 0.026392177367180844, 'depth': 9, 'l2_leaf_reg': 0.001974170660535803}


In [14]:
# Best trial의 파라미터
best_params = study2.best_trial.params

# 최적의 모델 학습
best_catboost = CatBoostRegressor(iterations=best_params['iterations'],
                                  learning_rate=best_params['learning_rate'],
                                  depth=best_params['depth'],
                                  l2_leaf_reg=best_params['l2_leaf_reg'],
                                  random_seed=42,
                                  loss_function='RMSE',
                                  eval_metric='RMSE',
                                  verbose=False)
# 범주형 데이터 'quality' : CatBoost의 cat_features 매개변수에 전달 필요
cat_features = ['quality']
best_catboost.fit(x_train, y_train)

# 저장 경로 설정
import pickle
import os
save_path = 'CatBoostRegressor_best_model.pkl'

# 모델을 'CatBoostRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_catboost, f)

# 저장된 모델 로드
with open('CatBoostRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 12708407674.759
rmse : 112731.574
r2 : 0.893


## RandomForestRegressor 를 이용한 가격 예측

In [15]:
import optuna
from sklearn.ensemble import RandomForestRegressor

def objective(trial):
    # Define the search space for each parameter
    n_estimators = trial.suggest_int('n_estimators', 100, 4000)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', [1.0, 'sqrt', 'log2'])
    
    # 모델 학습
    random_f2 = RandomForestRegressor(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     max_features=max_features,
                                     random_state=42)
    random_f2.fit(x_train, y_train)
    
    # 예측
    y_pred = random_f2.predict(x_test)
    
    # 모델 평가    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"mse : {mse}")
    print(f"rmse : {rmse}")
    print(f"r2 : {r2}")
    
    return mse

# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study.optimize(objective, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-06-21 02:27:35,831] A new study created in memory with name: no-name-673c16e2-f060-4684-810a-b8ffed9540b9
[I 2023-06-21 02:27:50,068] Trial 0 finished with value: 43716286396.29059 and parameters: {'n_estimators': 3268, 'max_depth': 2, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 43716286396.29059.


mse : 43716286396.29059
rmse : 209084.40017440467
r2 : 0.6330914293187853


[I 2023-06-21 02:28:21,084] Trial 1 finished with value: 13689536898.640268 and parameters: {'n_estimators': 3839, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 1 with value: 13689536898.640268.


mse : 13689536898.640268
rmse : 117002.29441613643
r2 : 0.8851044123181963


[I 2023-06-21 02:28:34,168] Trial 2 finished with value: 14318528224.480425 and parameters: {'n_estimators': 1946, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 1 with value: 13689536898.640268.


...

mse : 12955056790.495232
rmse : 113820.28286072404
r2 : 0.8912688665499754


[I 2023-06-21 03:19:41,740] Trial 98 finished with value: 12874565454.798836 and parameters: {'n_estimators': 721, 'max_depth': 13, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 96 with value: 12846638022.736105.


mse : 12874565454.798836
rmse : 113466.14232800389
r2 : 0.8919444262410449


[I 2023-06-21 03:19:42,859] Trial 99 finished with value: 12914861023.876562 and parameters: {'n_estimators': 104, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 96 with value: 12846638022.736105.


mse : 12914861023.876562
rmse : 113643.57009473331
r2 : 0.891606227577026
Best trial: score 12846638022.736105, 
params {'n_estimators': 1039, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}


In [16]:
# Best trial의 파라미터
best_params = study.best_trial.params

# 최적의 모델 학습
best_rf = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                max_depth=best_params['max_depth'],
                                min_samples_split=best_params['min_samples_split'],
                                min_samples_leaf=best_params['min_samples_leaf'],
                                max_features=best_params['max_features'],
                                random_state=42)
best_rf.fit(x_train, y_train)

# 저장
import pickle
import os

# 저장 경로 설정
save_path = 'RandomForestRegressor_best_model.pkl'

# 모델을 'RandomForestRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_rf, f)

# 모델 불러오기
# 저장된 모델을 로드
with open('RandomForestRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 12846638022.736
rmse : 113343.010
r2 : 0.892


## 카메라 6개 열 삭제 후 xgboost 를 이용한 가격 예측

In [17]:
remove_camera_df = prepro_df.copy()


# 카메라 삭제
remove_camera_df.drop(['back_camera_1', 'back_camera_2', 'back_camera_3',
                       'back_camera_number_1', 'back_camera_number_2',
                       'back_camera_number_3'], axis=1, inplace=True)

# 학습 데이터, 테스트 데이터 분리
x = remove_camera_df.loc[:, remove_camera_df.columns != 'price']
y = remove_camera_df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

In [18]:
# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study5 = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study5.optimize(objectiveXGB, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study5.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study5.best_trial.value, study5.best_trial.params))

[I 2023-06-21 03:23:26,766] A new study created in memory with name: no-name-66f9fcad-3952-4ce7-bbbe-012cecab5823
[I 2023-06-21 03:23:34,198] Trial 0 finished with value: 22428490158.55355 and parameters: {'n_estimators': 2098, 'learning_rate': 0.8672653087784747, 'max_depth': 4, 'min_child_weight': 7, 'gamma': 0.09294674528382324, 'subsample': 0.7895424832046107, 'colsample_bytree': 0.5544740072068335}. Best is trial 0 with value: 22428490158.55355.


mse : 22428490158.55355
rmse : 149761.444165558
r2 : 0.8117588215976452


[I 2023-06-21 03:24:10,972] Trial 1 finished with value: 20308239993.853394 and parameters: {'n_estimators': 3985, 'learning_rate': 0.4847435899174792, 'max_depth': 9, 'min_child_weight': 4, 'gamma': 0.38780268361377707, 'subsample': 0.6818725562320377, 'colsample_bytree': 0.7670200125128568}. Best is trial 1 with value: 20308239993.853394.


mse : 20308239993.853394
rmse : 142506.98226351364
r2 : 0.8295539735088733


[I 2023-06-21 03:24:20,357] Trial 2 finished with value: 15702187357.075865 and parameters: {'n_estimators': 1154, 'learning_rate': 0.11637413766677948, 'max_depth': 7, 'min_child_weight': 8, 'gamma': 0.13450575724422065, 'subsample': 0.5290135447600336, 'colsample_bytree': 0.9966015373707424}. Best is trial 2 with value: 15702187357.075865.


...

mse : 12125458840.458286
rmse : 110115.66119521004
r2 : 0.8982316399962109


[I 2023-06-21 03:32:39,581] Trial 98 finished with value: 12120977122.120275 and parameters: {'n_estimators': 1104, 'learning_rate': 0.01160247709122311, 'max_depth': 7, 'min_child_weight': 9, 'gamma': 0.44135966186887043, 'subsample': 0.5628559751501184, 'colsample_bytree': 0.6549916612367797}. Best is trial 88 with value: 12065316758.057497.


mse : 12120977122.120275
rmse : 110095.30926483778
r2 : 0.8982692548305243


[I 2023-06-21 03:32:45,839] Trial 99 finished with value: 12281682870.448175 and parameters: {'n_estimators': 1126, 'learning_rate': 0.013197747179941988, 'max_depth': 8, 'min_child_weight': 9, 'gamma': 0.45985063211005606, 'subsample': 0.5398170235596814, 'colsample_bytree': 0.6587601834699843}. Best is trial 88 with value: 12065316758.057497.


mse : 12281682870.448175
rmse : 110822.7542991428
r2 : 0.8969204596495995
Best trial: score 12065316758.057497, 
params {'n_estimators': 1160, 'learning_rate': 0.010446365347115766, 'max_depth': 6, 'min_child_weight': 10, 'gamma': 0.4838526274623721, 'subsample': 0.5484813233467711, 'colsample_bytree': 0.7025201506606376}


In [19]:
# Best trial의 파라미터
best_params = study5 .best_trial.params

# 최적의 모델 학습
best_xgb = XGBRegressor(n_estimators=best_params['n_estimators'],
                        learning_rate=best_params['learning_rate'],
                        max_depth=best_params['max_depth'],
                        min_child_weight=best_params['min_child_weight'],
                        gamma=best_params['gamma'],
                        subsample=best_params['subsample'],
                        colsample_bytree=best_params['colsample_bytree'],
                        random_state=42)
best_xgb.fit(x_train, y_train)

# 저장 경로 설정
import pickle
import os
save_path = 'remove_camera_XGBRegressor_best_model.pkl'

# 모델을 'XGBRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_xgb, f)

# 저장된 모델 로드
with open('remove_camera_XGBRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 12065316758.057
rmse : 109842.236
r2 : 0.899


## xgboost 를 이용한 가격 예측 (price 도 모두 스케일링 진행)

In [1]:
from phone_CrawlingDataPreprocessor import DataPreprocessor
prepro = DataPreprocessor("C:/Python_practice/DL_project/final_project/no_Yongsan_yes_Doksan/DA/dataset/phone_input_data.csv")
another_try_df = prepro.execute()
another_try_df

,price,quality,favoritecount,viewcount,product_name,screen_size,cost,capacity,ram,weight,Max_video_playtime,Max_audio_playtime,battery_capacity,hours,used_year
8137,450000,2.0,0.0,0.000000,아이폰 12 Mini,13.70,1160000,256,4,0.133,10,50,2227,0.016667,3.0
9748,970000,2.0,0.0,0.000000,갤럭시 Z 폴드4,19.21,1998700,256,12,0.263,20,72,4400,0.016667,1.0
21572,530000,3.0,0.0,0.000308,아이폰 12,15.40,1020000,128,4,0.162,11,65,2815,0.016667,3.0
19929,950000,4.0,0.0,0.000002,갤럭시 S 23 플러스,16.65,1353000,256,8,0.195,27,84,4700,0.033333,0.0
22773,560000,3.0,0.0,0.000217,아이폰 13 Mini,13.70,950000,128,4,0.140,13,55,2406,0.050000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,750000,4.0,0.0,0.000053,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,18,83,4500,7920.000000,3.0
9744,750000,2.0,0.0,0.000161,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,18,83,4500,8640.000000,3.0
12807,1050000,2.0,0.0,0.000308,갤럭시 Z 폴드2,19.30,2398000,256,12,0.282,18,83,4500,8640.000000,3.0
12796,1200000,2.0,0.0,0.000092,갤럭시 Z 폴드3,19.19,1998700,256,12,0.271,18,63,4400,8640.000000,2.0


In [2]:
import numpy as np
import pandas as pd
from all_x_scaling_and_processor import scaling_all_of_x
another_try_df = scaling_all_of_x(another_try_df)

In [6]:
# 학습 데이터, 테스트 데이터 분리
x = another_try_df.loc[:, another_try_df.columns != 'price']
y = another_try_df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

In [8]:
import optuna
from xgboost import XGBRegressor

def objectiveXGB(trial):
    # Define the search space for each parameter
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
    }

    # 모델 학습
    model = XGBRegressor(**params)
    model.fit(x_train, y_train)

    # 예측
    y_pred = model.predict(x_test)

    # 모델 평가    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"mse : {mse}")
    print(f"rmse : {rmse}")
    print(f"r2 : {r2}")
    
    return mse

# optimization study 진행
# direction: score 값을 최대 or 최소로 하는 방향으로 지정
study6 = optuna.create_study(direction='minimize')

# n_trials: 시도 횟수 (미입력시 key interrupt가 있을 때까지 무한 반복)
study6.optimize(objectiveXGB, n_trials=100)

# 모든 트라이얼이 완료되었는지 확인
if study6.best_trial is None:
    print("No trials completed.")
else:
    # 최적의 트라이얼 출력
    print('Best trial: score {}, \nparams {}'.format(study6.best_trial.value, study6.best_trial.params))

[I 2023-06-22 01:14:43,899] A new study created in memory with name: no-name-e11bdb34-3233-4260-8cc0-8499c91389cf
[I 2023-06-22 01:14:44,527] Trial 0 finished with value: 0.007100006328554453 and parameters: {'n_estimators': 761, 'learning_rate': 0.6820532613407668, 'max_depth': 1, 'min_child_weight': 1, 'gamma': 0.2693987685146609, 'subsample': 0.693130336376866, 'colsample_bytree': 0.7143149543416705}. Best is trial 0 with value: 0.007100006328554453.


mse : 0.007100006328554453
rmse : 0.08426153528481696
r2 : 0.8464574773771835


[I 2023-06-22 01:14:44,751] Trial 1 finished with value: 0.006745483215298172 and parameters: {'n_estimators': 160, 'learning_rate': 0.4416093326309871, 'max_depth': 3, 'min_child_weight': 4, 'gamma': 0.478543097108204, 'subsample': 0.8338324268648037, 'colsample_bytree': 0.5793908024620729}. Best is trial 1 with value: 0.006745483215298172.


mse : 0.006745483215298172
rmse : 0.08213089074945049
r2 : 0.8541242836613613


[I 2023-06-22 01:15:02,722] Trial 2 finished with value: 0.005638622544469938 and parameters: {'n_estimators': 3193, 'learning_rate': 0.6878552841233114, 'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.3348358272038441, 'subsample': 0.8010276144150819, 'colsample_bytree': 0.7796585092511125}. Best is trial 2 with value: 0.005638622544469938.


...

mse : 0.004871752449949667
rmse : 0.06979794015549218
r2 : 0.894645000843055


[I 2023-06-22 01:34:18,233] Trial 98 finished with value: 0.0048650086931245305 and parameters: {'n_estimators': 1852, 'learning_rate': 0.14892409402329998, 'max_depth': 10, 'min_child_weight': 6, 'gamma': 0.09647706706391859, 'subsample': 0.7095242155034165, 'colsample_bytree': 0.9749561959884427}. Best is trial 92 with value: 0.004805341907501084.


mse : 0.0048650086931245305
rmse : 0.06974961428656455
r2 : 0.8947908392250183


[I 2023-06-22 01:34:28,485] Trial 99 finished with value: 0.004860021086646523 and parameters: {'n_estimators': 1684, 'learning_rate': 0.0848352514111709, 'max_depth': 9, 'min_child_weight': 7, 'gamma': 0.06942515036904391, 'subsample': 0.7480271734155848, 'colsample_bytree': 0.9999600224225257}. Best is trial 92 with value: 0.004805341907501084.


mse : 0.004860021086646523
rmse : 0.06971385146903392
r2 : 0.8948986996472553
Best trial: score 0.004805341907501084, 
params {'n_estimators': 2039, 'learning_rate': 0.030840429680088564, 'max_depth': 10, 'min_child_weight': 5, 'gamma': 0.0612751843561137, 'subsample': 0.6802754404923279, 'colsample_bytree': 0.9590190429366244}


In [9]:
# Best trial의 파라미터
best_params = study6.best_trial.params

# 최적의 모델 학습
best_xgb = XGBRegressor(n_estimators=best_params['n_estimators'],
                        learning_rate=best_params['learning_rate'],
                        max_depth=best_params['max_depth'],
                        min_child_weight=best_params['min_child_weight'],
                        gamma=best_params['gamma'],
                        subsample=best_params['subsample'],
                        colsample_bytree=best_params['colsample_bytree'],
                        random_state=42)
best_xgb.fit(x_train, y_train)

# 저장 경로 설정
import pickle
import os
save_path = 'sixth_try_XGBRegressor_best_model.pkl'

# 모델을 'XGBRegressor_best_model.pkl' 파일로 저장
with open(save_path, 'wb') as f:
    pickle.dump(best_xgb, f)

# 저장된 모델 로드
with open('sixth_try_XGBRegressor_best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 로드된 모델로 예측
y_pred = loaded_model.predict(x_test)

# 모델 평가
from evaluate_score import evaluate_score
evaluate_score(y_test, y_pred)

mse : 0.005
rmse : 0.069
r2 : 0.896


In [ ]:
# 역변환 참고

# 데이터의 평균과 표준편차
self.mean_price = origin_df['price'].mean()
self.std_price = origin_df['price'].std()
# 6번째 모델 로그변환, 정규화 역변환

# 로그 변환된 price를 원래의 price로 변환
self.predicted_price = np.exp(self.predicted_price)

# 역변환
self.predicted_price = self.predicted_price * self.std_price + self.mean_price